In [ ]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

In [ ]:
openai.organization = os.getenv("ORGANIZATION_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Sample API Call
# response = openai.Completion.create(
#     engine="text-davinci-002",  # Specify the engine (model) you want to use
#     prompt="Translate the following English text to French: 'Hello, how are you?'",
#     max_tokens=50,  # Limit the length of the generated text
# )